In [1]:
import numpy as np
import librosa
from tensorflow.keras import models
import sounddevice as sd
import os
sr = 22100

2021-07-25 16:54:17.081563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sahil/myfiles/auv/training/assing_1/work/devel/lib:/opt/ros/noetic/lib
2021-07-25 16:54:17.081604: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [36]:
def play(sound,sr):
    sd.play(sound,sr)
    sd.wait()

In [3]:
def extract_cnn_feature(file):
    # print(file.shape)
    result = np.array([])
    mfcc = np.mean(librosa.feature.mfcc(file,sr=sr,n_mfcc=20).T,axis=0)
    result = np.hstack((result,mfcc))
    spec_centroid = np.mean(librosa.feature.spectral_centroid(file,sr).T,axis=0)
    result = np.hstack((result,spec_centroid))
    spec_bandwidth = np.mean(librosa.feature.spectral_bandwidth(file,sr).T,axis=0)
    result = np.hstack((result,spec_bandwidth))
    spec_contrast = np.mean(librosa.feature.spectral_contrast(file,sr).T,axis=0)
    result=np.hstack((result,spec_contrast))
    spec_flat = np.mean(librosa.feature.spectral_flatness(file).T,axis=0)
    result=np.hstack((result,spec_flat))
    return result

In [14]:
def extract_mlp_feature(file):
    result = np.array([])
    mfcc = np.mean(librosa.feature.mfcc(file, sr = sr, n_mfcc = 40).T, axis = 0)
    result = np.hstack((result, mfcc))
    chroma = np.mean(librosa.feature.chroma_stft(file, sr=sr).T, axis = 0)
    result = np.hstack((result, chroma))
    mel = np.mean(librosa.feature.melspectrogram(file, sr=sr).T, axis = 0)
    result = np.hstack((result, mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr = sr).T, axis = 0)
    result=np.hstack((result, spectral_rolloff))
    return result

In [47]:
def predict(vo,model,mean,std,sr,callback):

    voice = vo.copy()
    feat = callback(voice)
    feat = (feat-mean)/std
    max_ = np.amax(feat)
    min_ = np.amin(feat)
    feat = (2*feat-(max_ + min_))/(max_-min_)
    feat = np.expand_dims(feat,axis=0)
    feat = np.expand_dims(feat,axis=2)

    a = model.predict(feat)
    a = np.argmax(a)

    return a
    

In [13]:
cnn_model = models.load_model('../cnn_model/cnn_model_team-4')
mean_cnn = np.loadtxt('../cnn_model/mean_cnn.csv',delimiter=',')
std_cnn = np.loadtxt('../cnn_model/std_cnn.csv',delimiter=',')

gender_classifier = models.load_model('../mlp_model/Gender_classification_mlp_model_Team-4')
mean_mlp = np.loadtxt('../mlp_model/mean_mlp.csv')
std_mlp = np.loadtxt('../mlp_model/std_mlp.csv')

emotions = ['neutral','calm','happy','sad','angry','fearful','disgust','surprised']
gender = ['female','male']


In [52]:
path = 'sample_voices/'
live = os.listdir(path)
os.remove('predictions.txt')
fil = open('predictions.txt','w')
for i,v in enumerate(live):

    voice = np.zeros([120000])
    v,_ = librosa.load(path+v)
    voice[:v.shape[0]] = v

    result = predict(voice,cnn_model,mean_cnn,std_cnn,sr,extract_cnn_feature)
    g = predict(voice,gender_classifier,mean_mlp,std_mlp,sr,extract_mlp_feature)
    fil.seek(0)
    fil.writelines(['Voice-'+str(i+1)+'\n','MODEL PREDICTIONS- \n','Emotion = '+emotions[result]+'\n','Gender = '+gender[g]])
    fil.truncate()
    fil.flush()
    play(voice,sr)

ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8526:(snd_pcm_recover) underrun o